## Final Pipeline for all drugs using PCA
**Issues**
* Dobutamine and Isoprenaline have Accuracy of 1.00 🤙🏼🤙🏼

### Initialise the dataframes

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
PCA_df = pd.read_csv("FeaturesPCA.csv",index_col=[0])
PCA_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.523476,-0.238780,0.704734,-0.251208,-0.704963,0.083239,-0.016321,-0.338960,-0.101965,-0.320476,...,0.015840,0.138977,-0.090525,0.044719,-0.039023,-0.074408,0.051322,0.048233,-0.095420,-0.117281
1,0.170270,-0.465586,-0.305915,-0.404152,-0.197291,0.284409,-0.234281,-0.308266,0.124785,-0.735082,...,-0.052570,0.086144,0.077468,-0.026543,-0.035970,-0.196330,-0.088867,-0.180740,-0.027363,0.073743
2,0.654504,0.359562,-0.358608,-0.289298,-0.006726,-0.024271,-0.087419,0.293584,-0.030858,-0.270481,...,-0.064232,-0.082718,0.154134,-0.006525,-0.048231,-0.018546,0.037308,0.100942,-0.006716,0.212511
3,-0.045029,0.771255,0.576467,-0.112582,-0.453351,-0.281525,0.753308,-0.256638,-0.458042,0.048076,...,-0.001842,-0.047436,-0.093628,-0.014237,0.117447,0.159251,-0.136388,-0.017576,0.007260,0.066017
4,-0.753907,-0.061010,-0.256266,-0.156085,0.005529,0.046611,-0.245375,-0.039222,-0.447446,0.192799,...,0.234802,0.002370,0.107162,0.069440,0.113330,0.017976,-0.023594,-0.014171,-0.010660,0.057907


In [3]:
df = pd.read_csv("drug_onehot_latest.csv")
df

,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,870258,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0
1999,870646,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0
2000,879601,1,1,1,1,0,1,1,0,0,0,1,1,1,0,1,0,0,0
2001,905163,0,1,1,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0


In [4]:
df.drop(['inpatient.number'], axis=1,inplace=True)

### Initialise the Neural net and The pipeline

In [5]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(128, input_dim=50
                           , activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=50,verbose = 0)
    else:
        model.fit(X_train, y_train, epochs=50, class_weight = weights)
    
    model.evaluate(X_test, y_test,verbose = 0)
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [6]:
def model(q, trainer):

    count_class_0, count_class_1 = trainer[q].value_counts()[0], trainer[q].value_counts()[1]

    # Divide by class
    df_class_0 = trainer[trainer[q] == 0]
    df_class_1 = trainer[trainer[q] == 1]
    if count_class_0>count_class_1:
        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
    else:
        df_class_0_over = df_class_0.sample(count_class_1,replace=True)
        df_test_over = pd.concat([df_class_0_over, df_class_1], axis=0)

    print('Random over-sampling:')
    print(df_test_over[q].value_counts())
    X = df_test_over.drop(q,axis='columns')
    y = df_test_over[q]

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
    # loss = keras.losses.BinaryCrossentropy()
    # weights = -1
    y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)
    

In [7]:
# q = "Benazepril hydrochloride tablet"

In [8]:
# PCA_df = pd.concat([PCA_df, df[q]], axis = 1)

In [9]:
# PCA_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.523476,-0.238780,0.704734,-0.251208,-0.704963,0.083239,-0.016321,-0.338960,-0.101965,-0.320476,...,0.015840,0.138977,-0.090525,0.044719,-0.039023,-0.074408,0.051322,0.048233,-0.095420,-0.117281
1,0.170270,-0.465586,-0.305915,-0.404152,-0.197291,0.284409,-0.234281,-0.308266,0.124785,-0.735082,...,-0.052570,0.086144,0.077468,-0.026543,-0.035970,-0.196330,-0.088867,-0.180740,-0.027363,0.073743
2,0.654504,0.359562,-0.358608,-0.289298,-0.006726,-0.024271,-0.087419,0.293584,-0.030858,-0.270481,...,-0.064232,-0.082718,0.154134,-0.006525,-0.048231,-0.018546,0.037308,0.100942,-0.006716,0.212511
3,-0.045029,0.771255,0.576467,-0.112582,-0.453351,-0.281525,0.753308,-0.256638,-0.458042,0.048076,...,-0.001842,-0.047436,-0.093628,-0.014237,0.117447,0.159251,-0.136388,-0.017576,0.007260,0.066017
4,-0.753907,-0.061010,-0.256266,-0.156085,0.005529,0.046611,-0.245375,-0.039222,-0.447446,0.192799,...,0.234802,0.002370,0.107162,0.069440,0.113330,0.017976,-0.023594,-0.014171,-0.010660,0.057907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,0.607132,-0.667063,0.378551,0.666605,-0.797473,0.081307,0.031920,-0.160044,0.410042,0.017275,...,-0.062490,-0.148070,0.087492,-0.051457,-0.051467,0.006208,-0.153569,0.029043,0.092999,-0.093636
1999,-0.722724,-0.220347,-0.190170,-0.248624,-0.109251,0.008667,0.206417,-0.177188,0.251510,0.281932,...,0.051737,0.013974,0.103063,0.049904,-0.107219,-0.110953,-0.047693,0.014594,0.023851,-0.007725
2000,0.428533,0.346765,-0.332385,-0.350568,-0.034702,-0.058557,-0.004901,0.033122,0.451865,-0.406711,...,-0.050924,-0.122386,0.019130,-0.109956,0.025977,0.039704,-0.030960,-0.165412,0.020573,0.076900
2001,-0.467465,0.661539,-0.149135,-0.295775,0.015919,0.000456,0.062080,-0.130210,0.400708,-0.272297,...,-0.153761,0.124958,-0.060721,-0.052512,-0.050559,0.174951,0.038165,-0.030763,-0.098479,-0.099807


### Run the Pipeline

In [10]:
for q in df.columns:
    print("Running for :",q)
    temp = pd.concat([PCA_df, df[q]], axis = 1)
    model(q,temp)


Running for : Atorvastatin calcium tablet
Random over-sampling:
0    1185
1    1185
Name: Atorvastatin calcium tablet, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.82      0.70      0.76       237
           1       0.74      0.85      0.79       237

    accuracy                           0.77       474
   macro avg       0.78      0.77      0.77       474
weighted avg       0.78      0.77      0.77       474

Running for : Benazepril hydrochloride tablet
Random over-sampling:
0    1569
1    1569
Name: Benazepril hydrochloride tablet, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.82      0.87       314
           1       0.84      0.95      0.89       314

    accuracy                           0.88       628
   macro avg       0.89      0.88      0.88       628
weighted avg       0.89      0.88      0.88       628

Running for : Deslanosid